<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #12:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px; text-align: center;">
        VAR, Causalidad de Granger e Impulso Respuesta
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Noviembre 2024
</i>

<hr>

# 1. Preparación de Datos y Selección de Rezagos

## Configuración Inicial
- Importar datos desde archivo .dta
- Configurar serie temporal

```stata
clear all
use "LAB_VAR.dta", clear
tsset fecha
```

## Test de Razón de Verosimilitud
```stata
* Determinación del número máximo de rezagos
* 4 [(39/4)^(1/4)] = 7.06, entonces M = 8
gen lags = .
gen LR_stat = .
forval m = 1/8 {
    quietly varbasic INF DES TPM, lags(1/`m') nograph
    matrix Sigma_r = e(Sigma)
    quietly varbasic INF DES TPM, lags(1/`=`m'+1') nograph
    matrix Sigma_u = e(Sigma)
    gen LR = 39*(ln(det(Sigma_r))-ln(det(Sigma_u)))
    replace lags = `m' in `m'
    replace LR_stat = LR in `m'
    drop LR
}
```

> 🤓 **Nota**: El valor crítico chi-cuadrado con 9 grados de libertad al 5% es 16.919

# 2. Modelo VAR(1) y Causalidad de Granger

## Estimación del VAR
Basados en el análisis previo, estimamos un VAR(1):

```stata
varbasic INF DES TPM, lags(1/1) step(10) irf
```

## Prueba de Causalidad de Granger
```stata
vargranger
```

> 🔍 **Resultado**: La TPM causa en el sentido de Granger al Desempleo (DES)

# 3. Funciones de Impulso-Respuesta

## Análisis de Diferentes Ordenamientos
El orden de las variables afecta los resultados:

```stata
* Crear modelos con diferentes ordenamientos
irf create Modelo_1, step(10) replace
irf create Modelo_2, order(TPM DES INF) step(10) replace
irf create Modelo_3, order(TPM INF DES) step(10) replace
irf create Modelo_4, order(INF TPM DES) step(10) replace
irf create Modelo_5, order(DES INF TPM) step(10) replace
irf create Modelo_6, order(DES TPM INF) step(10) replace
```


## Análisis de Impulso-Respuesta
```stata
* Graficar impulso-respuesta específicos
irf graph oirf, impulse(TPM) response(INF DES TPM) yline(0,lcolor(black) lpattern(dash)) byopts(yrescale) xlabel(0(2)10)
```

> 📊 **Observación**: Un shock de una desviación estándar en la TPM tiene un efecto positivo gradual sobre el desempleo

# 4. Descomposición de la Varianza


```stata
* Análisis de la descomposición de varianza
use Modelo.irf, clear
keep if irfname == "Modelo_5"
keep if response == "INF"
keep fevd step impulse
reshape wide fevd, i(step) j(impulse) string
graph bar (asis) fevdTPM fevdINF fevdDES, over(step) stack
```

## Matriz de Descomposición de Cholesky (SW2001)
Consideraciones importantes:
1. Desempleo y TPM no tienen efectos contemporáneos en la inflación (precios rígidos)
2. El desempleo responde con rezago a los cambios en la TPM. Inflación sí tiene efectos contemporáneos en el desempleo. (Curva Phillips)
3. Shocks en inflación y desempleo afectan contemporáneamente la TPM (Banco Central tiene información de la economía para establecer la TPM)

$
\Omega y_t =
\begin{bmatrix}
b_{11} & 0 & 0 \\
b_{21} & b_{22} & 0 \\
b_{31} & b_{32} & b_{33}
\end{bmatrix}
\begin{bmatrix}
INF_t \\
DES_t \\
TPM_{t}
\end{bmatrix}
= B y_{t-1} + \varepsilon_t
$